In [47]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib as plt


# Importing Data

### Metadata about the articles

In [48]:
#Articles is the name of all the articles in our dataset, the index serves also as an id to retrieve articles in other files
# the names are url encoded
articles_df = pd.read_csv("./data/wikispeedia_paths-and-graph/articles.tsv",names = ["name"] ,skiprows=12,sep="\t")
#The categories of the articles, row index is the same as for the articles df
categories_df = pd.read_csv("./data/wikispeedia_paths-and-graph/categories.tsv",names = ["article","category"],skiprows=13, sep="\t")
categories_df = categories_df.groupby(["article"]).agg(lambda x : list(x))
#each row of of the links df contains the list of article names that an article links to
links_df = pd.read_csv("./data/wikispeedia_paths-and-graph/links.tsv",skiprows=12, names = ["from","to"],sep="\t")
out_links_df = links_df.groupby(["from"]).agg(lambda x : list(x))
in_links_df = links_df.groupby(["to"]).agg(lambda x : list(x))
#
shortest_path_distance = pd.read_csv("./data/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt",names=["distances"], skiprows=17)



print("Articles")
display(articles_df)
print("Categories")
display(categories_df)
print("Links")
display(links_df)
print("shortest_path_distance")
display(shortest_path_distance)


Articles


,name
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in
1,%C3%85land
2,%C3%89douard_Manet
3,%C3%89ire
4,%C3%93engus_I_of_the_Picts
...,...
4599,Zionism
4600,Zirconium
4601,Zoroaster
4602,Zuid-Gelders


Categories


,category
article,
%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,[subject.History.British_History.British_Histo...
%C3%85land,"[subject.Countries, subject.Geography.European..."
%C3%89douard_Manet,[subject.People.Artists]
%C3%89ire,"[subject.Countries, subject.Geography.European..."
%C3%93engus_I_of_the_Picts,[subject.History.British_History.British_Histo...
...,...
Zionism,"[subject.People.Political_People, subject.Reli..."
Zirconium,[subject.Science.Chemistry.Chemical_elements]
Zoroaster,[subject.People.Religious_figures_and_leaders]


Links


,from,to
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Bede
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Columba
2,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,D%C3%A1l_Riata
3,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Great_Britain
4,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Ireland
...,...,...
119877,Zulu,South_Africa
119878,Zulu,Swaziland
119879,Zulu,United_Kingdom
119880,Zulu,Zambia


shortest_path_distance


,distances
0,0_____33333325634333435_2433544334_3_422343544...
1,_0____22222325623232424_2422544324_3_312242544...
2,__0___33222425623232324_2333444433_3_422343434...
3,___0__33333325634233334_2433434333_2_423343433...
4,____0_22323335633332435_2433545434_3_423343544...
...,...
4599,______22222325622231424_1322544334_3_422232544...
4600,______33333434523232434_2332544324_3_323333544...
4601,______22222424522231434_2322545434_3_422232544...
4602,______33333436733342435_2433545444_3_523353544...


### Played games dataset

In [49]:
paths_finished_df = pd.read_csv("./data/wikispeedia_paths-and-graph/paths_finished.tsv",names = ["hashedIpAddress","timestamp","durationInSec","path","rating"],skiprows=16, sep="\t")
paths_finished_df["path"] = paths_finished_df["path"].apply(lambda x: x.split(";"))
paths_finished_df["start"] = paths_finished_df["path"].apply(lambda x: x[0])
paths_finished_df["target"] = paths_finished_df["path"].apply(lambda x: x[-1])
paths_finished_df["path"] = paths_finished_df["path"].apply(lambda x: x[1:-1])
paths_unfinished_df = pd.read_csv("./data/wikispeedia_paths-and-graph/paths_unfinished.tsv",names=["hashedIpAddress",   "timestamp ",  "durationInSec",  "path",   "target",   "type"],skiprows=17, sep="\t")
paths_unfinished_df["path"] = paths_unfinished_df["path"].apply(lambda x: x.split(";"))
paths_unfinished_df["start"] = paths_unfinished_df["path"].apply(lambda x: x[0])
paths_unfinished_df["path"] = paths_unfinished_df["path"].apply(lambda x: x[1:])

print("finished paths")
display(paths_finished_df)
print("unfinished paths")
display(paths_unfinished_df)

finished paths


,hashedIpAddress,timestamp,durationInSec,path,rating,start,target
0,6a3701d319fc3754,1297740409,166,"[15th_century, 16th_century, Pacific_Ocean, At...",NaN,14th_century,African_slave_trade
1,3824310e536af032,1344753412,88,"[Europe, Africa, Atlantic_slave_trade]",3.0,14th_century,African_slave_trade
2,415612e93584d30e,1349298640,138,"[Niger, Nigeria, British_Empire, Slavery, Afri...",NaN,14th_century,African_slave_trade
3,64dd5cd342e3780c,1265613925,37,"[Renaissance, Ancient_Greece]",NaN,14th_century,Greece
4,015245d773376aab,1366730828,175,"[Italy, Roman_Catholic_Church, HIV, Ronald_Rea...",3.0,14th_century,John_F._Kennedy
...,...,...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,[Ancient_Egypt],NaN,Yagan,Civilization
51314,2ef7ac844cefda58,1300254138,165,"[Folklore, Brothers_Grimm, <, 19th_century, Ch...",3.0,Yagan,Fiction
51315,12863abb7887f890,1385095372,228,"[Australia, England, France, United_States, Te...",NaN,Yagan,U.S._Open_%28tennis%29
51316,19f8284371753362,1298792567,56,"[Australia, United_States]",1.0,Yarralumla%2C_Australian_Capital_Territory,Abraham_Lincoln


unfinished paths


,hashedIpAddress,timestamp,durationInSec,path,target,type,start
0,2426091a53125110,1297054935,1804,[],Microsoft,timeout,Obi-Wan_Kenobi
1,26141fd878806294,1297055651,1805,[],Caracas,timeout,Julius_Caesar
2,2b015fb8181c48f2,1297090819,1818,"[Democracy, Alexander_the_Great]",First_Crusade,timeout,Malawi
3,53a53bc244e08a6a,1297094761,49,[],Mount_St._Helens,restart,Paraguay
4,53a53bc244e08a6a,1297099105,1808,[Bolivia],Mount_St._Helens,timeout,Paraguay
...,...,...,...,...,...,...,...
24870,109ed71f571d86e9,1389787605,180,"[Tuberculosis, World_Health_Organization, Mala...",Cholera,restart,Franz_Kafka
24871,232f992e57d43e8d,1389787697,6,[],Hollandic,restart,Modern_history
24872,2e09a7224600a7cd,1389798400,1900,"[Linguistics, Culture, Popular_culture, <]",The_Beatles,timeout,Computer_programming
24873,60af9e2138051b96,1389799481,1903,"[United_Kingdom, World_War_II, Battle_of_Midway]",Alan_Turing,timeout,Jamaica


In [50]:
out_num_links = out_links_df["to"].apply(lambda x: len(x))
display(out_num_links[out_num_links==(out_num_links.max())])

in_num_links = in_links_df["from"].apply(lambda x: len(x))
display(in_num_links[in_num_links==(in_num_links.max())])

from
United_States    294
Name: to, dtype: int64

to
United_States    1551
Name: from, dtype: int64

In [51]:
#list of articles that were visited without the start and finish
visited_nodes_without_sf=paths_finished_df["path"].explode().value_counts()
visited_nodes_without_sf

<                     20561
United_States          8824
Europe                 4319
United_Kingdom         3860
Earth                  3153
                      ...  
Wikinews                  1
Zebra_shark               1
Gray_reef_shark           1
Blue_shark                1
Arcadia_%28play%29        1
Name: path, Length: 3407, dtype: int64

In [52]:
visited_nodes_without_sf=paths_unfinished_df["path"].explode().value_counts()
visited_nodes_without_sf

<                                  12907
United_States                       3546
United_Kingdom                      1418
Europe                              1234
England                             1181
                                   ...  
Michigan_State_University              1
Ubuntu_%28Linux_distribution%29        1
Lake_Kivu                              1
Palmyra_Atoll                          1
2005_Sumatra_earthquake                1
Name: path, Length: 3190, dtype: int64